In [1]:
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.3/julia-1.3.1-linux-x86_64.tar.gz' -O julia.tar.gz
    tar -x -f julia.tar.gz -C /usr/local --strip-components 1
    rm julia.tar.gz
fi
    julia -e '\
        using Pkg                                                          ;\
        Pkg.update()                                                       ;\
        Pkg.add("IJulia")                                                  ;\
        Pkg.add("BenchmarkTools")                                          ;\
        Pkg.add("CuArrays")                                                ;\
        Pkg.add("BenchmarkTools")                                          ;\
        Pkg.add("DataFrames")                                              ;\
        Pkg.add("CSV")                                                     ;\
        Pkg.precompile()                                                   ;\'
echo 'Done'

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.ju

# DataFrames.jl 介紹 (二): Joins 與 Split-Apply-Combine Strategy



## Day 018 作業1：The Split-Apply-Combine Strategy

同 Day 017，請載入 2020/4/2 COVID-19 Daily Report 資料集

In [2]:
# 彙總 DataFrame 資訊

!julia -e '\
using DataFrames, CSV                                                                                                                         ;\
url =  "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-02-2020.csv"       ;\
df = CSV.read(download(url))                                                                                                                  ;\
println(describe(df))                                                                                                                         ;\'

12×8 DataFrame
│ Row │ variable       │ mean     │ min           │ median   │ max                     │ nunique │ nmissing │ eltype                  │
│     │ Symbol         │ Union…   │ Any           │ Union…   │ Any                     │ Union…  │ Union…   │ Type                    │
├─────┼────────────────┼──────────┼───────────────┼──────────┼─────────────────────────┼─────────┼──────────┼─────────────────────────┤
│ 1   │ FIPS           │ 30958.4  │ 66            │ 29093.0  │ 99999                   │         │ 281      │ Union{Missing, Int64}   │
│ 2   │ Admin2         │          │ Abbeville     │          │ unassigned              │ 1431    │ 264      │ Union{Missing, String}  │
│ 3   │ Province_State │          │ Alabama       │          │ Zhejiang                │ 135     │ 177      │ Union{Missing, String}  │
│ 4   │ Country_Region │          │ Afghanistan   │          │ Zimbabwe                │ 181     │          │ String                  │
│ 5   │ Last_Update    │         

計算計算每個國家/地區的累計確診人數

【提示】可使用 groupby 相關的函式

In [3]:
# 顯示所有資料

!julia -e '\
using DataFrames, CSV                                                                                                                         ;\
url =  "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-02-2020.csv"       ;\
df = CSV.read(download(url))                                                                                                                  ;\
println(by(df, :Country_Region, :Confirmed => sum ))                                                                                          ;\'   

181×2 DataFrame
│ Row │ Country_Region                   │ Confirmed_sum │
│     │ String                           │ Int64         │
├─────┼──────────────────────────────────┼───────────────┤
│ 1   │ US                               │ 243599        │
│ 2   │ Canada                           │ 11284         │
│ 3   │ United Kingdom                   │ 34173         │
│ 4   │ China                            │ 82432         │
│ 5   │ Netherlands                      │ 14788         │
│ 6   │ Australia                        │ 5116          │
│ 7   │ Denmark                          │ 3573          │
│ 8   │ France                           │ 59929         │
│ 9   │ Afghanistan                      │ 273           │
│ 10  │ Albania                          │ 277           │
│ 11  │ Algeria                          │ 986           │
│ 12  │ Andorra                          │ 428           │
│ 13  │ Angola                           │ 8             │
│ 14  │ Antigua and Barbuda             

## 作業2：請列出美國各區域或事件 (依 Province_State 欄) 的累積確診、死亡、康復人數。

【提示】使用 Split-Apply-Combine Strategy 時，有幾種不同的函式可以使用，請參照 `DataFrames.jl` 官方文件 [https://juliadata.github.io/DataFrames.jl/stable/lib/functions/#DataFrames.groupby](https://juliadata.github.io/DataFrames.jl/stable/lib/functions/#DataFrames.groupby)

In [4]:
!julia -e '\
using DataFrames, CSV                                                                                                                         ;\
url =  "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-02-2020.csv"       ;\
df = CSV.read(download(url))                                                                                                                  ;\
println(by( df[df.Country_Region .== "US", :] , :Province_State , [:Confirmed => sum , :Deaths => sum , :Recovered => sum  ]  ))              ;\'      

58×4 DataFrame
│ Row │ Province_State           │ Confirmed_sum │ Deaths_sum │ Recovered_sum │
│     │ Union{Missing, String}   │ Int64         │ Int64      │ Int64         │
├─────┼──────────────────────────┼───────────────┼────────────┼───────────────┤
│ 1   │ South Carolina           │ 1554          │ 31         │ 0             │
│ 2   │ Louisiana                │ 9149          │ 310        │ 0             │
│ 3   │ Virginia                 │ 1706          │ 41         │ 0             │
│ 4   │ Idaho                    │ 776           │ 9          │ 0             │
│ 5   │ Iowa                     │ 614           │ 11         │ 0             │
│ 6   │ Missouri                 │ 1857          │ 22         │ 0             │
│ 7   │ Oklahoma                 │ 881           │ 34         │ 0             │
│ 8   │ Colorado                 │ 3342          │ 80         │ 0             │
│ 9   │ Illinois                 │ 7695          │ 163        │ 0             │
│ 10  │ Indiana          